In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 16712 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0

Setting up libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Setting up unattended-upgrades (0.98ubuntu1.1) ...

Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) 

In [5]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [7]:
!ls ./drive/datasets/

labeledTrainData.tsv  testData.tsv  unlabeledTrainData.tsv


In [0]:
import pylab
import calendar
import numpy as np
import pandas as pd
import seaborn as sn
from scipy import stats
import re
#import missingno as msno
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [0]:
train = pd.read_csv("./drive/datasets/labeledTrainData.tsv", header=0,delimiter="\t", quoting=3)
test = pd.read_csv("./drive/datasets/testData.tsv", header=0, delimiter="\t",quoting=3 )
unlabeled_train = pd.read_csv("./drive/datasets/unlabeledTrainData.tsv", header=0, delimiter="\t",quoting=3 )

In [14]:
print(len(train))
print(len(test))
print(len(unlabeled_train))

25000
25000
50000


In [15]:
train["review"][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [0]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [21]:
import nltk.data
nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


In [22]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. 

Parsing sentences from unlabeled set


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behin

In [25]:
!pip install gensim

    100% |████████████████████████████████| 22.6MB 59kB/s 
    100% |████████████████████████████████| 133kB 7.9MB/s 
    100% |████████████████████████████████| 1.4MB 908kB/s 
    100% |████████████████████████████████| 4.1MB 312kB/s 
    100% |████████████████████████████████| 61kB 9.8MB/s 
    100% |████████████████████████████████| 552kB 2.3MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /content/.cache/pip/wheels/cf/54/36/b003d8c2d26aadffc21f0677009ed53cf9575a97fc71fbba76
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /content/.cache/pip/wheels/31/9c/20/996d65ca104cbca940b1b053299b68459391c01c774d073126
Successfully built smart-open bz2file


In [26]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-04-02 21:34:31,235 : INFO : 'pattern' package not found; tag filters are not available for English
2018-04-02 21:34:31,244 : INFO : collecting all words and their counts
2018-04-02 21:34:31,245 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-02 21:34:31,317 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2018-04-02 21:34:31,391 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


Training model...


2018-04-02 21:34:31,467 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2018-04-02 21:34:31,540 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2018-04-02 21:34:31,612 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2018-04-02 21:34:31,684 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2018-04-02 21:34:31,766 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2018-04-02 21:34:31,841 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2018-04-02 21:34:31,914 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2018-04-02 21:34:31,986 : INFO : PROGRESS: at sentence #100000, processed 2226967 words, keeping 50207 word types
2018-04-02 21:34:32,059 : INFO : PROGRESS: at sentence #110000, processed 2446581 words, keeping 

2018-04-02 21:34:34,547 : INFO : PROGRESS: at sentence #440000, processed 9821225 words, keeping 94906 word types
2018-04-02 21:34:34,625 : INFO : PROGRESS: at sentence #450000, processed 10044987 words, keeping 96036 word types
2018-04-02 21:34:34,704 : INFO : PROGRESS: at sentence #460000, processed 10277747 words, keeping 97088 word types
2018-04-02 21:34:34,787 : INFO : PROGRESS: at sentence #470000, processed 10505672 words, keeping 97933 word types
2018-04-02 21:34:34,864 : INFO : PROGRESS: at sentence #480000, processed 10726056 words, keeping 98862 word types
2018-04-02 21:34:34,942 : INFO : PROGRESS: at sentence #490000, processed 10952800 words, keeping 99871 word types
2018-04-02 21:34:35,017 : INFO : PROGRESS: at sentence #500000, processed 11174456 words, keeping 100765 word types
2018-04-02 21:34:35,093 : INFO : PROGRESS: at sentence #510000, processed 11399731 words, keeping 101699 word types
2018-04-02 21:34:35,170 : INFO : PROGRESS: at sentence #520000, processed 11623

2018-04-02 21:34:37,489 : INFO : sample=0.001 downsamples 48 most-common words
2018-04-02 21:34:37,490 : INFO : downsampling leaves estimated 12749798 word corpus (74.0% of prior 17239125)
2018-04-02 21:34:37,540 : INFO : estimated required memory for 16490 words and 300 dimensions: 47821000 bytes
2018-04-02 21:34:37,542 : INFO : resetting layer weights
2018-04-02 21:34:37,845 : INFO : training model with 4 workers on 16490 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2018-04-02 21:34:38,887 : INFO : EPOCH 1 - PROGRESS: at 2.56% examples, 319883 words/s, in_qsize 6, out_qsize 1
2018-04-02 21:34:39,910 : INFO : EPOCH 1 - PROGRESS: at 5.37% examples, 334706 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:34:40,910 : INFO : EPOCH 1 - PROGRESS: at 8.06% examples, 334998 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:34:41,919 : INFO : EPOCH 1 - PROGRESS: at 10.76% examples, 336443 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:34:42,920 : INFO : EPOCH 1 - P

2018-04-02 21:35:15,459 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-02 21:35:15,473 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-02 21:35:15,481 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-02 21:35:15,482 : INFO : EPOCH - 1 : training on 17798270 raw words (12751468 effective words) took 37.6s, 338917 effective words/s
2018-04-02 21:35:16,524 : INFO : EPOCH 2 - PROGRESS: at 2.62% examples, 325321 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:35:17,543 : INFO : EPOCH 2 - PROGRESS: at 5.36% examples, 334281 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:35:18,568 : INFO : EPOCH 2 - PROGRESS: at 8.16% examples, 336708 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:35:19,571 : INFO : EPOCH 2 - PROGRESS: at 10.87% examples, 338115 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:35:20,579 : INFO : EPOCH 2 - PROGRESS: at 13.60% examples, 338771 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:35:21,580 :

2018-04-02 21:35:52,925 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-02 21:35:52,949 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-02 21:35:52,952 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-02 21:35:52,954 : INFO : EPOCH - 2 : training on 17798270 raw words (12747507 effective words) took 37.5s, 340267 effective words/s
2018-04-02 21:35:53,985 : INFO : EPOCH 3 - PROGRESS: at 2.51% examples, 316214 words/s, in_qsize 8, out_qsize 1
2018-04-02 21:35:54,994 : INFO : EPOCH 3 - PROGRESS: at 5.25% examples, 331827 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:35:56,017 : INFO : EPOCH 3 - PROGRESS: at 8.06% examples, 335340 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:35:57,039 : INFO : EPOCH 3 - PROGRESS: at 10.81% examples, 337266 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:35:58,072 : INFO : EPOCH 3 - PROGRESS: at 13.60% examples, 338293 words/s, in_qsize 7, out_qsize 1
2018-04-02 21:35:59,105 :

2018-04-02 21:36:30,533 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-02 21:36:30,551 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-02 21:36:30,557 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-02 21:36:30,558 : INFO : EPOCH - 3 : training on 17798270 raw words (12748533 effective words) took 37.6s, 339138 effective words/s
2018-04-02 21:36:31,573 : INFO : EPOCH 4 - PROGRESS: at 2.56% examples, 328221 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:36:32,591 : INFO : EPOCH 4 - PROGRESS: at 5.25% examples, 332691 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:36:33,605 : INFO : EPOCH 4 - PROGRESS: at 8.11% examples, 339201 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:36:34,649 : INFO : EPOCH 4 - PROGRESS: at 10.81% examples, 336678 words/s, in_qsize 6, out_qsize 1
2018-04-02 21:36:35,651 : INFO : EPOCH 4 - PROGRESS: at 13.60% examples, 339434 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:36:36,674 :

2018-04-02 21:37:07,750 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-02 21:37:07,770 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-02 21:37:07,772 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-02 21:37:07,776 : INFO : EPOCH - 4 : training on 17798270 raw words (12750645 effective words) took 37.2s, 342713 effective words/s
2018-04-02 21:37:08,802 : INFO : EPOCH 5 - PROGRESS: at 2.56% examples, 323065 words/s, in_qsize 5, out_qsize 2
2018-04-02 21:37:09,839 : INFO : EPOCH 5 - PROGRESS: at 5.36% examples, 334400 words/s, in_qsize 7, out_qsize 1
2018-04-02 21:37:10,874 : INFO : EPOCH 5 - PROGRESS: at 8.22% examples, 337605 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:37:11,878 : INFO : EPOCH 5 - PROGRESS: at 10.92% examples, 338742 words/s, in_qsize 6, out_qsize 1
2018-04-02 21:37:12,880 : INFO : EPOCH 5 - PROGRESS: at 13.65% examples, 339633 words/s, in_qsize 7, out_qsize 0
2018-04-02 21:37:13,881 :

2018-04-02 21:37:45,148 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-02 21:37:45,173 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-02 21:37:45,178 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-02 21:37:45,180 : INFO : EPOCH - 5 : training on 17798270 raw words (12748630 effective words) took 37.4s, 340909 effective words/s
2018-04-02 21:37:45,184 : INFO : training on a 88991350 raw words (63746783 effective words) took 187.3s, 340278 effective words/s
2018-04-02 21:37:45,190 : INFO : precomputing L2-norms of word weight vectors
2018-04-02 21:37:45,366 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2018-04-02 21:37:45,368 : INFO : not storing attribute vectors_norm
2018-04-02 21:37:45,369 : INFO : not storing attribute cum_table
2018-04-02 21:37:45,806 : INFO : saved 300features_40minwords_10context


In [27]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [31]:
model.doesnt_match("man king queen boy".split())

'queen'